In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

# 웹드라이버 설정 (Chrome 기준)
driver = webdriver.Chrome()

# missing_names.csv에서 '업체' 컬럼 가져오기
df = pd.read_csv('missing_names.csv', encoding='cp949')
keywords = df['업체명'].tolist()

# 베이스 URL
base_url = "https://m.search.naver.com/search.naver?sm=mtp_sly.hst&where=m&query="

def get_reviews_from_url(url):
    driver.get(url)
    
    # 방문자 리뷰 버튼 클릭
    try:
        review_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/review/visitor')]"))
        )
        review_button.click()
        time.sleep(2)  # 페이지 로딩 대기
    except TimeoutException:
        print("방문자 리뷰 버튼을 찾을 수 없습니다.")
        return []
    except NoSuchElementException:
        print("방문자 리뷰 버튼을 찾을 수 없습니다.")
        return []

    driver.switch_to.window(driver.window_handles[-1])

    # 리뷰 더보기 클릭 함수
    def click_more_button():
        try:
            more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "TeItc"))
            )
            more_button.click()
            return True
        except TimeoutException:
            return False
        except NoSuchElementException:
            return False

    # 리뷰 텍스트 수집 함수
    def get_reviews():
        reviews = []
        review_elements = driver.find_elements(By.CLASS_NAME, "zPfVt")
        for review in review_elements:
            reviews.append(review.text)
        return reviews

    # 최대 100개의 리뷰 수집
    all_reviews = []
    while len(all_reviews) < 100:
        current_reviews = get_reviews()
        if current_reviews:
            all_reviews.extend(current_reviews)
            all_reviews = list(set(all_reviews))  # 중복 제거
            print(f"현재 수집된 리뷰 개수: {len(all_reviews)}")
        else:
            break
        
        if not click_more_button():
            break
        time.sleep(1)  # 페이지 로딩 대기

    return all_reviews[:100]

# 각 키워드에 대해 리뷰 수집 및 CSV 저장
for idx, keyword in enumerate(keywords):
    query_url = base_url + keyword
    reviews = get_reviews_from_url(query_url)
    if reviews:
        reviews_df = pd.DataFrame(reviews, columns=["Review"])
        file_name = f"reviews_{idx+1}.csv"
        reviews_df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"{file_name} 파일에 {len(reviews)}개의 리뷰가 저장되었습니다.")
    else:
        print(f"{keyword}의 리뷰를 수집하지 못했습니다.")

# 웹드라이버 종료
driver.quit()


현재 수집된 리뷰 개수: 10
현재 수집된 리뷰 개수: 20
현재 수집된 리뷰 개수: 30
현재 수집된 리뷰 개수: 40
현재 수집된 리뷰 개수: 50
현재 수집된 리뷰 개수: 60
현재 수집된 리뷰 개수: 70
현재 수집된 리뷰 개수: 80
현재 수집된 리뷰 개수: 90
현재 수집된 리뷰 개수: 100
reviews_1.csv 파일에 100개의 리뷰가 저장되었습니다.
현재 수집된 리뷰 개수: 10
현재 수집된 리뷰 개수: 20
현재 수집된 리뷰 개수: 30
현재 수집된 리뷰 개수: 40
현재 수집된 리뷰 개수: 50
현재 수집된 리뷰 개수: 60
현재 수집된 리뷰 개수: 70
현재 수집된 리뷰 개수: 80
현재 수집된 리뷰 개수: 90
현재 수집된 리뷰 개수: 95
reviews_2.csv 파일에 95개의 리뷰가 저장되었습니다.
현재 수집된 리뷰 개수: 10
현재 수집된 리뷰 개수: 20
현재 수집된 리뷰 개수: 30
현재 수집된 리뷰 개수: 40
현재 수집된 리뷰 개수: 50
현재 수집된 리뷰 개수: 60
현재 수집된 리뷰 개수: 70
현재 수집된 리뷰 개수: 80
현재 수집된 리뷰 개수: 90
현재 수집된 리뷰 개수: 100
reviews_3.csv 파일에 100개의 리뷰가 저장되었습니다.
